# Import

In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# number 1 to 10 data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/home/arg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Define layers amd some functions

In [2]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    inital = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(inital)

def bias_variable(shape):
    inital = tf.constant(0.1, shape=shape)
    return tf.Variable(inital)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# Create model

## Tensorboard : with

## Placeholder

In [3]:
# define placeholder for inputs to network
with tf.name_scope('Inputs'):
    xs = tf.placeholder(tf.float32, [None, 784]) # 28x28
    x_image = tf.reshape(xs, [-1, 28, 28, 1])
with tf.name_scope('Labels'):
    ys = tf.placeholder(tf.float32, [None, 10])

keep_prob = tf.placeholder(tf.float32)


## Add layers 

In [7]:
with tf.name_scope('model'):
    ## conv1 layer ##
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32]) #patch 5x5, in channel size 1, out size 32
        b_conv1 = bias_variable([32])
        #activation function
        with tf.name_scope('relu'):
            h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) #output size 28x28x32

    ## pool1 layer ##
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1) #output size 14x14x32


    ###############You can add your own layers if you want################
    #Here is an example of another layers:conv2+pool2
    '''
    ## conv2 layer ##
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64]) 
        b_conv2 = bias_variable([64])
        #activation function
        with tf.name_scope('relu'):
            h_conv1 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

    ## pool2 layer ##
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2) 
    
    '''   
    ######################################################################
    with tf.name_scope('flatten'):
        '''If you've added layers above make sure that your flatten layer has attached to the last layer.
        For the example given above, the code below should be like:
        h_flat = tf.contrib.layers.flatten(h_pool2)'''
        h_flat = tf.contrib.layers.flatten(h_pool1)

    ## fc1 layer ##
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable([h_flat.shape[1].value, 1024])
        b_fc1 = bias_variable([1024])
        with tf.name_scope('relu'):
            h_fc1 = tf.nn.relu(tf.matmul(h_flat, W_fc1) + b_fc1)
        with tf.name_scope('dropout'):
            h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    ## output layer ##
    with tf.name_scope('softmax'):
        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([10])
        prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# Loss function

In [8]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))
    tf.summary.scalar("Loss", cross_entropy)

# GradientDescent

In [9]:
with tf.name_scope('GradientDescent'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Accuracy

In [10]:
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(ys, 1))
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('Accuracy', acc)

# Start training and recording in Tensorboard

In [11]:
################################ These are the codes for recording the graph to Tensorboard#########################

logs_path = 'TensorBoard/'
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(logs_path , graph = tf.get_default_graph())

############################################## Training ############################################################

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(500):
    batch_xs, batch_ys = mnist.train.next_batch(25)
    sess.run(train_step, feed_dict = {xs: batch_xs, ys: batch_ys, keep_prob:0.5} )
    if i % 50 == 0:
        summary = sess.run(merged, feed_dict = {xs: batch_xs, ys: batch_ys, keep_prob:0.5})
        writer.add_summary(summary, i)
        print(compute_accuracy(mnist.test.images, mnist.test.labels))
        
sess.close()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.1091
0.6729
0.7801
0.8442
0.8745
0.881
0.8909
0.8946
0.9094
0.8993


#  Open tensorboard

## $ tensorboard --logdir='TensorBoard/'